# Azure AI

이 워크샵에서는 Microsoft Agent Framework를 사용하여 Azure AI 에이전트를 만드는 방법을 학습합니다.

## 목차
1. [환경 설정](#환경-설정)
2. [기본 에이전트](#기본-에이전트)
3. [함수 도구 사용](#함수-도구-사용)
4. [스레드 관리](#스레드-관리)
5. [코드 인터프리터](#코드-인터프리터)
6. [파일 검색](#파일-검색)
7. [Bing 검색 연동](#bing-검색-연동)
8. [MCP 도구 사용](#mcp-도구-사용)
9. [여러 도구 통합](#여러-도구-통합)

## 사전 요구사항
- Azure 구독
- Azure AI 프로젝트
- Azure CLI 로그인 (`az login`)
- 필요한 환경 변수 설정

## 환경 설정

먼저 필요한 라이브러리를 임포트하고 환경 변수를 로드합니다.

In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv

# override=True로 설정하여 기존 환경 변수를 덮어씁니다
load_dotenv(override=True)

print("환경 변수가 로드되었습니다.")

In [ ]:
# 필수 라이브러리 임포트
import asyncio
import os
from random import randint
from typing import Annotated
from datetime import datetime, timezone
from pathlib import Path

from agent_framework import ChatAgent, AgentThread, AgentRunResponse, ChatResponseUpdate
from agent_framework import HostedCodeInterpreterTool, HostedFileSearchTool, HostedVectorStoreContent
from agent_framework import HostedWebSearchTool, HostedMCPTool
from agent_framework_azure_ai import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from pydantic import Field

print("라이브러리 임포트 완료!")

## 1. 기본 에이전트

가장 간단한 Azure AI 에이전트를 만들어봅니다. 스트리밍과 일반 응답 방식을 모두 테스트합니다.

In [ ]:
# 날씨 도구 함수 정의
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."

print("날씨 도구 함수가 정의되었습니다.")

In [ ]:
# 비스트리밍 방식 예제
async def non_streaming_example():
    print("=== 비스트리밍 응답 예제 ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="WeatherAgent",
            instructions="You are a helpful weather agent.",
            tools=get_weather,
        ) as agent,
    ):
        query = "What's the weather like in Seattle?"
        print(f"사용자: {query}")
        result = await agent.run(query)
        print(f"에이전트: {result}\n")

await non_streaming_example()

In [ ]:
# 스트리밍 방식 예제
async def streaming_example():
    print("=== 스트리밍 응답 예제 ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="WeatherAgent",
            instructions="You are a helpful weather agent.",
            tools=get_weather,
        ) as agent,
    ):
        query = "What's the weather like in Portland?"
        print(f"사용자: {query}")
        print("에이전트: ", end="", flush=True)
        async for chunk in agent.run_stream(query):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        print("\n")

await streaming_example()

## 2. 함수 도구 사용

여러 함수 도구를 에이전트에 추가하여 더 강력한 기능을 제공합니다.

In [ ]:
# 시간 도구 함수 정의
def get_time() -> str:
    """Get the current UTC time."""
    current_time = datetime.now(timezone.utc)
    return f"The current UTC time is {current_time.strftime('%Y-%m-%d %H:%M:%S')}."

print("시간 도구 함수가 정의되었습니다.")

In [ ]:
# 에이전트 레벨에서 도구 정의
async def tools_on_agent_level():
    print("=== 에이전트 레벨 도구 정의 ===")
    
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(async_credential=credential),
            instructions="You are a helpful assistant that can provide weather and time information.",
            tools=[get_weather, get_time],
        ) as agent,
    ):
        query = "What's the weather in Tokyo and what time is it?"
        print(f"사용자: {query}")
        result = await agent.run(query)
        print(f"에이전트: {result}\n")

await tools_on_agent_level()

In [ ]:
# 런타임에 도구 전달
async def tools_on_run_level():
    print("=== 런타임 도구 전달 ===")
    
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(async_credential=credential),
            instructions="You are a helpful assistant.",
        ) as agent,
    ):
        query = "What's the weather in New York?"
        print(f"사용자: {query}")
        result = await agent.run(query, tools=get_weather)
        print(f"에이전트: {result}\n")

await tools_on_run_level()

## 3. 스레드 관리

대화 컨텍스트를 유지하기 위한 스레드 관리 방법을 학습합니다.

In [ ]:
# 스레드 지속성 예제
async def thread_persistence_example():
    print("=== 스레드 지속성 예제 ===")
    
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(async_credential=credential),
            instructions="You are a helpful weather agent.",
            tools=get_weather,
        ) as agent,
    ):
        # 새 스레드 생성
        thread = agent.get_new_thread()
        
        # 첫 번째 대화
        query1 = "What's the weather in London?"
        print(f"사용자: {query1}")
        result1 = await agent.run(query1, thread=thread, store=True)
        print(f"에이전트: {result1}\n")
        
        # 두 번째 대화 (같은 스레드 사용)
        query2 = "What about Paris?"
        print(f"사용자: {query2}")
        result2 = await agent.run(query2, thread=thread, store=True)
        print(f"에이전트: {result2}\n")
        
        # 세 번째 대화 (컨텍스트 유지 확인)
        query3 = "Which city had better weather?"
        print(f"사용자: {query3}")
        result3 = await agent.run(query3, thread=thread, store=True)
        print(f"에이전트: {result3}\n")

await thread_persistence_example()

## 4. 코드 인터프리터

Python 코드를 실행할 수 있는 코드 인터프리터 도구를 사용합니다.

In [ ]:
# 코드 인터프리터 예제
async def code_interpreter_example():
    print("=== 코드 인터프리터 예제 ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as chat_client,
    ):
        agent = chat_client.create_agent(
            name="CodingAgent",
            instructions="You are a helpful assistant that can write and execute Python code to solve problems.",
            tools=HostedCodeInterpreterTool(),
        )
        
        query = "Generate the factorial of 100 using python code, show the code and execute it."
        print(f"사용자: {query}")
        response = await AgentRunResponse.from_agent_response_generator(agent.run_stream(query))
        print(f"에이전트: {response}\n")

await code_interpreter_example()

## 5. Bing 검색 연동

Bing 검색을 통해 실시간 웹 정보를 활용합니다.

**참고:** Bing 연결이 Azure AI 프로젝트에 설정되어 있어야 합니다.

In [ ]:
# Bing 검색 예제
async def bing_search_example():
    print("=== Bing 검색 예제 ===")
    
    # Bing 검색 도구 생성
    bing_search_tool = HostedWebSearchTool(
        name="Bing Grounding Search",
        description="Search the web for current information using Bing",
    )
    
    async with (
        AzureAIAgentClient(async_credential=AzureCliCredential()) as client,
        ChatAgent(
            chat_client=client,
            name="BingSearchAgent",
            instructions=(
                "You are a helpful assistant that can search the web for current information. "
                "Use the Bing search tool to find up-to-date information and provide accurate, "
                "well-sourced answers. Always cite your sources when possible."
            ),
            tools=bing_search_tool,
        ) as agent,
    ):
        user_input = "What is the most popular programming language?"
        print(f"사용자: {user_input}")
        response = await agent.run(user_input)
        print(f"에이전트: {response.text}\n")

# 주석 해제하여 실행 (Bing 연결이 필요합니다)
# await bing_search_example()

## 6. MCP 도구 사용

Model Context Protocol (MCP) 서버를 통해 외부 도구를 통합합니다.

In [ ]:
# Hosted MCP 예제
async def hosted_mcp_example():
    print("=== Hosted MCP 예제 ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as chat_client,
    ):
        agent = chat_client.create_agent(
            name="DocsAgent",
            instructions="You are a helpful assistant that can help with microsoft documentation questions.",
            tools=HostedMCPTool(
                name="Microsoft Learn MCP",
                url="https://learn.microsoft.com/api/mcp",
            ),
        )
        
        query = "How to create an Azure storage account using az cli?"
        print(f"사용자: {query}")
        result = await agent.run(query)
        print(f"에이전트: {result}\n")

await hosted_mcp_example()

## 7. 여러 도구 통합

여러 종류의 도구를 하나의 에이전트에서 함께 사용합니다.

In [ ]:
# 여러 도구 통합 예제
async def multiple_tools_example():
    print("=== 여러 도구 통합 예제 ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as chat_client,
    ):
        agent = chat_client.create_agent(
            name="MultiToolAgent",
            instructions="You are a helpful assistant with multiple capabilities.",
            tools=[
                HostedMCPTool(
                    name="Microsoft Learn MCP",
                    url="https://learn.microsoft.com/api/mcp",
                ),
                get_time,
                get_weather,
            ],
        )
        
        query = "What time is it and what's the weather in Seoul?"
        print(f"사용자: {query}")
        result = await agent.run(query)
        print(f"에이전트: {result}\n")

await multiple_tools_example()

## 8. 명시적 설정 사용

환경 변수 대신 명시적으로 설정을 지정하는 방법입니다.

In [ ]:
# 명시적 설정 예제
async def explicit_settings_example():
    print("=== 명시적 설정 예제 ===")
    
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(
                project_endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
                model_deployment_name=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
                async_credential=credential,
                agent_name="WeatherAgent",
            ),
            instructions="You are a helpful weather agent.",
            tools=get_weather,
        ) as agent,
    ):
        result = await agent.run("What's the weather like in New York?")
        print(f"결과: {result}\n")

await explicit_settings_example()

## 요약

이 워크샵에서 다룬 내용:

1. ✅ **기본 에이전트**: 스트리밍/비스트리밍 응답
2. ✅ **함수 도구**: 날씨, 시간 조회 함수
3. ✅ **스레드 관리**: 대화 컨텍스트 유지
4. ✅ **코드 인터프리터**: Python 코드 실행
5. ✅ **Bing 검색**: 실시간 웹 검색
6. ✅ **MCP 도구**: 외부 도구 통합
7. ✅ **여러 도구**: 다중 도구 통합
8. ✅ **명시적 설정**: 환경 변수 없이 설정

## 다음 단계

- 파일 검색 도구로 문서 분석
- OpenAPI 도구 통합
- 사용자 승인 워크플로우
- 프로덕션 배포 패턴

## 참고 자료

- [Agent Framework 문서](https://github.com/microsoft/agent-framework)
- [Azure AI 문서](https://learn.microsoft.com/azure/ai-services/)
- [샘플 코드](../)